# Measuring Firms' Vertical Relatedness by IO table
## Method
1. Read IO table from Year 1992, 1997, 2002, 2007, 2012
2. Create IO coeffcient table
3. Create industry vertical relatedness table
4. Create SIC-IO conversion table
5. Build SIC-IO vertical relatedness function
6. Read Sales table
7. Calculate firm vertical relatedness based on sales

In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import os


## Functions

In [779]:
# function for industry vertical relatedness
def indvr(df):
    df['Commodity I-O code'] = df['Commodity I-O code'].apply(str).apply(lambda x: x.strip())
    df['Industry I-O code'] = df['Industry I-O code'].apply(str).apply(lambda x: x.strip())
    df = df.set_index(['Commodity I-O code','Industry I-O code'])
    df = df.unstack().fillna(0)
    dfco = df/df.sum() # coeffcient table
    dfco = dfco.rename(columns={'input': "input per"}, level=0)
    tt = dfco.stack()
    res = pd.DataFrame()
    for x , y in tt.index:
        if (y,x) in tt.index:
            x_y = tt.loc[x,y]['input per'] + tt.loc[y,x]['input per']
            index =  pd.MultiIndex.from_tuples([(x,y)], names=('Commodity I-O code', 'Industry I-O code'))
            temp = pd.DataFrame({'input per':x_y}, index=index)
            res = pd.concat([res, temp])
        else:
            x_y = tt.loc[x,y]['input per'] + 0
            index =  pd.MultiIndex.from_tuples([(x,y)], names=('Commodity I-O code', 'Industry I-O code'))
            temp = pd.DataFrame({'input per':x_y}, index=index)
            res = pd.concat([res, temp])
    dfVR = res
    dfVR['input per'] = dfVR['input per']/2
    dfVR.rename(columns={'input per': 'vertical relatedness'}, inplace=True)
    return dfVR

In [846]:
# function for finding vertical relatedness based on two SIC codes
def sicvr(s1, s2, dfVR, dfSic):
    import itertools
    if ( s1 in dfSic['SIC'].values) & (s2 in dfSic['SIC'].values):
        a = dfSic[dfSic['SIC']== s1]['digits6'].values
        b = dfSic[dfSic['SIC']== s2]['digits6'].values
        c = list(itertools.product(a, b))
        sicvr = dfVR.loc[list(set(c) & set(dfVR.index))].sum()/len(c)
    else:
        sicvr = pd.Series({'vertical relatedness':0})
    return sicvr   

In [845]:
# make sure data sort by gvkey fyear -salesm
# find firm year vertical relatedness by SIC coded segments
def firmsicvr(gvkey, fyear, dfss, dfVR, dfSic):
    dftt = dfss9296.loc[gvkey, fyear].copy()
    
    dftt['weight'] = dftt.groupby(['gvkey','fyear'])['salesm'].transform('sum')- dftt['salesm'].max()

    dftt['weight'] = dftt['salesm']/dftt['weight']

    dftt.iloc[0,-1]=0 #set max sales' sic weight 0


    dftt['sicpairvr']=dftt['SICS1'].apply(lambda x: sicvr(dftt['SICS1'][0],x, dfVR, dfSic))

    dftt['wsicvr']=dftt['weight']*dftt['sicpairvr']

    return dftt['wsicvr'].sum()

In [854]:
# create dataframe for firms' vertical relatedness based on SIC codes. 
def dffirmsicvr(dfss, dfVR, dfSic):
    dffirmvr = []
    for gvkey, fyear in dfss.index:
        dffirmvr.append(firmsicvr(gvkey, fyear, dfss, dfVR, dfSic))
    dffirmvr = pd.DataFrame({'firm vertical relatedness': dffirmvr}, index = dfss.index)
    return dffirmvr

In [851]:
# find IO industry code by NAICS code. 
def findIO(naics,dfnaics):
    if naics in dfnaics.index:
        io = dfnaics.loc[naics,['I-O Industry number']]
    elif naics[:5] in dfnaics.index:
        io = dfnaics.loc[naics[:5],['I-O Industry number']]
    elif naics[:4] in dfnaics.index:
        io = dfnaics.loc[naics[:4],['I-O Industry number']]
    elif naics[:3] in dfnaics.index:
        io = dfnaics.loc[naics[:3],['I-O Industry number']]
    elif naics[:2] in dfnaics.index:
        io = dfnaics.loc[naics[:2],['I-O Industry number']]
    else:
        io = pd.Series(dtype='object')
    return io

In [852]:
# find vertical relatedness by two NAICS codes
def naicsvr(n1, n2, dfVR, dfnaics):
    if (len(findIO(n1,dfnaics))>0) & (len(findIO(n2,dfnaics))>0):
        
        if set(itertools.product(findIO(n1,dfnaics), findIO(n2,dfnaics))) & set(dfVR.index)!= set(): 
            # if IO pairs from iput NACISs is not in dfVR table, return 0 
            naicsvr = dfVR.loc[list(set(itertools.product(findIO(n1,dfnaics), findIO(n2,dfnaics))) & set(dfVR.index))].sum()/len(set(itertools.product(findIO(n1,dfnaics), findIO(n2,dfnaics))))
        else: # if IO pairs from iput NACISs is not in dfVR table, return 0 
            naicsvr = pd.Series({'vertical relatedness':0})
    else:
        naicsvr = pd.Series({'vertical relatedness':0})
    return naicsvr

In [853]:
# make sure data sort by gvkey fyear -salesm
# find firm year vertical relatedness by NAICS coded segments
def firmnavr(gvkey, fyear, dfss, dfVR, dfnaics):
    dftt = dfss.loc[gvkey, fyear].copy()
    
    #dftt['weight'] = (dftt.groupby(['gvkey','fyear'])['salesm'].transform('sum')- dftt['salesm'].max())

    dftt['weight'] = dftt['salesm']/(dftt.groupby(['gvkey','fyear'])['salesm'].transform('sum')- dftt['salesm'].max())

    dftt.iloc[0,4]=0 #set max sales' sic weight 0


    dftt['naicspairvr']=dftt['NAICSS1'].apply(lambda x: naicsvr(dftt['NAICSS1'][0],x,dfVR,dfnaics))

    dftt['wsicvr']=dftt['weight']*dftt['naicspairvr']

    return dftt['wsicvr'].sum()

In [ ]:
# create dataframe for firms' vertical relatedness based on NAICS codes. 
def dffirmvr(dfss, dfVR, dfnaics):
    dffirmvr = []
    for gvkey, fyear in dfss.index:
        dffirmvr.append(firmnavr(gvkey, fyear, dfss, dfVR, dfnaics))
    dffirmvr = pd.DataFrame({'firm vertical relatedness': dffirmvr}, index = dfss.index)
    return dffirmvr

### IO Table 1992

In [728]:
df92 = pd.read_csv('IO1992/disk2/IOUse1992.csv')

In [729]:
df92

,Commodity I-O code,Industry I-O code,input
0,10100,10100,0
1,10100,10200,0
2,10100,10301,183
3,10100,10302,0
4,10100,20100,0
...,...,...,...
264153,900000,9930I5,0
264154,900000,9930I6,0
264155,900000,9930I7,0
264156,900000,9930I8,0


In [731]:
df92['Commodity I-O code'] = df92['Commodity I-O code'].apply(str).apply(lambda x: x.zfill(6))
df92['Industry I-O code'] = df92['Industry I-O code'].apply(str).apply(lambda x: x.zfill(6))

In [732]:
df92

,Commodity I-O code,Industry I-O code,input
0,010100,010100,0
1,010100,010200,0
2,010100,010301,183
3,010100,010302,0
4,010100,020100,0
...,...,...,...
264153,900000,9930I5,0
264154,900000,9930I6,0
264155,900000,9930I7,0
264156,900000,9930I8,0


In [733]:
df92VR=indvr(df92)

In [734]:
df92VR

vertical relatedness
Commodity I-O code Industry I-O code                      
010100             010100                         0.000000
                   010200                         0.000000
                   010301                         0.001756
                   010302                         0.000000
                   020100                         0.000000
...                                                    ...
900000             9930I5                         0.000000
                   9930I6                         0.000000
                   9930I7                         0.000000
                   9930I8                         0.000000
                   9930I9                         0.000000

[262194 rows x 1 columns]

#### SIC-IO Conversion Table

In [618]:
#import 1987 SIC-IO conversion table for year 1992
df92Sic = pd.read_csv('IO1992/disk2/IO_SIC_Conversion.csv', dtype={'digits6': str,'SIC':str  })

In [736]:
df92Sic.head()

,Unnamed: 0,header,digits2,digits6,digits6_des,dupli,notes,SICs,SIC_exclu,SIC,SIC_n
0,1,"AGRICULTURE, FORESTRY, AND FISHERIES",1 Livestock and livestock products,010100,Dairy farm products,False,NaN,"024,019, 0259, 029",NaN,0190,16.0
1,2,"AGRICULTURE, FORESTRY, AND FISHERIES",1 Livestock and livestock products,010100,Dairy farm products,False,NaN,"024,019, 0259, 029",NaN,0191,16.0
2,3,"AGRICULTURE, FORESTRY, AND FISHERIES",1 Livestock and livestock products,010100,Dairy farm products,False,NaN,"024,019, 0259, 029",NaN,0192,16.0
3,4,"AGRICULTURE, FORESTRY, AND FISHERIES",1 Livestock and livestock products,010100,Dairy farm products,False,NaN,"024,019, 0259, 029",NaN,0193,16.0
4,5,"AGRICULTURE, FORESTRY, AND FISHERIES",1 Livestock and livestock products,010100,Dairy farm products,False,NaN,"024,019, 0259, 029",NaN,0194,16.0


#### Calculated Firms' Vertical Relatedness Based on Sales

In [624]:
dfsicsale = pd.read_excel('sicsales.xlsx', dtype={'gvkey':str, 'SICS1':str}) # read  segment sales table from COMPUSTAT

In [761]:
dfsicsale.head()

,gvkey,SICS1,year,salesm,fyear
0,001004,5088,1977,46.139999,1976
1,001004,4582,1977,34.395000,1976
2,001006,2711,1976,6.260000,1976
3,001010,3743,1976,254.533997,1976
4,001010,3714,1976,139.679001,1976


In [626]:
### Sales 1992 - 1996 SIC table
dfsicsale[(dfsicsale['fyear']>=1992) & (dfsicsale['fyear']<=1996)]

,gvkey,SICS1,year,salesm,fyear
169635,001004,5088,1993,382.779999,1992
169636,001009,3462,1992,51.407001,1992
169637,001010,3743,1992,320.148010,1992
169638,001011,4841,1992,4.664000,1992
169639,001013,3661,1992,316.496002,1992
...,...,...,...,...,...
230937,223007,1381,1996,53.973999,1996
230938,223007,1389,1996,50.835999,1996
230939,223007,7372,1996,46.756001,1996
230940,223007,1389,1996,15.533000,1996


In [627]:
# select year 1992~1997 from segment sales table and set index gvkey fyear
dfss9296 = dfsicsale[(dfsicsale['fyear']>=1992) & (dfsicsale['fyear']<=1996)]

dfss9296 = dfss9296.set_index(['gvkey','fyear'])

In [663]:
# sort index
dfss9296 = dfss9296.sort_index()

#### Firm Vertical Relatedness from 1992 to 1996

In [775]:
#Firm level vertical relatedness for year 1992~1996
dffirmvr9296 = []
for gvkey, fyear in dfss9296.index:
    dffirmvr9296.append(firmsicvr(gvkey, fyear))
dffirmvr9296 = pd.DataFrame({'firm vertical relatedness': dffirmvr9296}, index = dfss9296.index)

In [855]:
dffirmvr9296 = dffirmsicvr(dfss9296, df92VR, df92Sic)

In [856]:
dffirmvr9296

firm vertical relatedness
gvkey  fyear                           
001004 1992                    0.000000
       1993                    0.000000
       1994                    0.000000
       1995                    0.000000
       1996                    0.000000
...                                 ...
223007 1996                    0.002516
       1996                    0.002516
277918 1994                    0.000000
       1995                    0.000000
       1996                    0.000000

[61307 rows x 1 columns]

In [778]:
## Export firm vertical relatedness 92-96 to csv
dffirmvr9296.to_csv('firmvr9296m.csv')

### IO table 1997

In [5]:
# make sure IO code 6 digits  index str
df97 = pd.read_csv('IO1997/IOUse1997.csv')

In [780]:
df97.head()

,Commodity I-O code,Industry I-O code,input
0,1111A0,1111A0,1363.5
1,1111A0,112100,13.8
2,1111A0,112A00,4.3
3,1111A0,311221,139.6
4,1111A0,311222,9175.6


In [7]:
df97['Commodity I-O code'] = df97['Commodity I-O code'].apply(str).apply(lambda x: x.strip())
df97['Industry I-O code'] = df97['Industry I-O code'].apply(str).apply(lambda x: x.strip())

In [782]:
df97VR=indvr(df97)

In [10]:
df97VR

vertical relatedness
Commodity I-O code Industry I-O code                      
1111A0             1111A0                         0.067311
                   112100                         0.000114
                   112A00                         0.000108
                   311221                         0.008504
                   311222                         0.338458
...                                                    ...
V00300             S00202                         0.221588
                   S00203                         0.112649
                   S00500                         0.079667
                   S00700                         0.500000
                   S00800                         0.337587

[78357 rows x 1 columns]

#### NAICS-IO Conversion Table 1997

In [16]:
df97naics = pd.read_csv('IO1997/IO-NACIS1997.csv', dtype={'I-O Industry number': str,'NAICS':str  })

In [396]:
df97naics.index

Index(['23', '23', '23', '23', '23', '23', '23', '23', '23', '23',
       ...
       '339994', '339995', '339999', '541511', '541512', '541513', '541519',
       '811191', '811192', '811198'],
      dtype='object', name='NACIS', length=579)

In [20]:
df97naics.set_index('NAICS',inplace=True)

In [801]:
df97naics

,I-O Industry number,I-O Industry title
NACIS,,
23,230110,"New residential 1-unit structures, nonfarm"
23,230120,"New multifamily housing structures, nonfarm"
23,230130,"New residential additions and alterations, non..."
23,230140,New farm housing units and additions and alter...
23,230210,Manufacturing and industrial buildings
...,...,...
541513,54151A,"Other computer related services, including fac..."
541519,54151A,"Other computer related services, including fac..."
811191,8111A0,"Automotive repair and maintenance, except car ..."


#### Calculate firm's vertical relatedness based on sales

In [870]:
dfnaics = pd.read_excel('naicssales.xlsx', dtype={'gvkey':str, 'SICS1':str, 'NAICSS1':str})

In [871]:
dfnaics.head()

,gvkey,fyear,year,salesm,SICS1,NAICSS1
0,001004,1992,1993,382.779999,5088,421860
1,001004,1993,1994,407.753998,5088,421860
2,001004,1994,1995,451.394989,5088,421860
3,001004,1995,1996,504.989990,5088,421860
4,001004,1996,1997,589.328003,5088,421860


In [872]:
dfnaics.sort_values(by=["gvkey","fyear","salesm"], ascending=[True,True,False],inplace=True)

In [873]:
dfnaics.set_index(['gvkey', 'fyear'], inplace=True)

In [874]:
dfnaics

year      salesm SICS1 NAICSS1
gvkey  fyear                                
001004 1992   1993  382.779999  5088  421860
       1993   1994  407.753998  5088  421860
       1994   1995  451.394989  5088  421860
       1995   1996  504.989990  5088  421860
       1996   1997  589.328003  5088  421860
...            ...         ...   ...     ...
330227 2019   2019    5.968000  4412  483111
331856 2019   2019    0.000000  2836  325414
332115 2019   2019    0.000000  2836  325414
335466 2018   2018   20.653999  4499  532411
       2019   2019   41.813000  4499  532411

[332711 rows x 4 columns]

#### Select firm fyear between 1997 and 2001

In [268]:
dfnaics.reset_index(inplace = True)

dfss9701 = dfnaics[(dfnaics['fyear']>=1997) & (dfnaics['fyear']<=2001)].copy()

dfss9701.set_index(['gvkey', 'fyear'], inplace=True)

dfss9701

In [807]:
firmnavr('001004', 2000,dfss9701, df97VR, df97naics)

0.018043475190552015

## Firm vertical relatedness from 1997 to 2001

In [809]:
dffirmvr9701 = dffirmvr(dfss9701, df97VR, df97naics)

In [810]:
dffirmvr9701

firm vertical relatedness
gvkey  fyear                           
001004 1997                    0.000000
       1998                    0.029610
       1998                    0.029610
       1998                    0.029610
       1999                    0.020134
...                                 ...
277918 1997                    0.000000
       1998                    0.000000
       1999                    0.000000
       2000                    0.000000
       2001                    0.000000

[74028 rows x 1 columns]

In [811]:
#### Export firm vertical relatedness 97-01 to csv
dffirmvr9701.to_csv('firmvr9701a.csv')

### IO Table 2002

In [332]:
df02 = pd.read_csv('IO2002/IOUse2002.csv')

In [812]:
df02.head()

,Commodity I-O code,Industry I-O code,input
0,1111A0,1111A0,1025.2
1,1111A0,1111B0,14.2
2,1111A0,111200,1.0
3,1111A0,111910,5.4
4,1111A0,112120,0.5


In [813]:
df02VR = indvr(df02)

In [814]:
df02VR

vertical relatedness
Commodity I-O code Industry I-O code                      
1111A0             1111A0                         0.070211
                   1111B0                         0.011218
                   111200                         0.000028
                   111335                         0.000000
                   1113A0                         0.000000
...                                                    ...
V00300             S00203                         0.114260
                   S00500                         0.078605
                   S00600                         0.053395
                   S00700                         0.036387
                   S00800                         0.304804

[189209 rows x 1 columns]

#### NAICS-IO Conversion Table 2002

In [816]:
df02naics = pd.read_csv('IO2002/IO-NAICS2002.csv', dtype={'I-O Industry number': str,'NAICS':str  })

In [817]:
df02naics

,I-O Industry number,I-O Industry title,NAICS
0,1111A0,Oilseed farming ...,11111
1,1111A0,Oilseed farming ...,11112
2,1111B0,Grain farming ...,11113
3,1111B0,Grain farming ...,11114
4,1111B0,Grain farming ...,11115
...,...,...,...
532,813A00,"Grantmaking, giving, and social advocacy organ...",8133
533,813B00,"Civic, social, professional, and similar organ...",8134
534,813B00,"Civic, social, professional, and similar organ...",8139
535,814000,Private households ...,814


In [818]:
# delete space in strings
df02naics['I-O Industry number'] = df02naics['I-O Industry number'].apply(str).apply(lambda x: x.strip())
df02naics['NAICS'] = df02naics['NAICS'].apply(str).apply(lambda x: x.strip())

In [819]:
df02naics.set_index('NAICS',inplace=True)

In [820]:
findIO('11211', df02naics)

I-O Industry number    1121A0
Name: 11211, dtype: object

#### Calculate firm's vertical relatedness based on sales

In [409]:
### Select firm year between 2002 and 2006
dfss0206 = dfnaics[(dfnaics['fyear']>=2002) & (dfnaics['fyear']<=2006)].copy()
dfss0206.set_index(['gvkey', 'fyear'], inplace=True)

In [821]:
dfss0206

year       salesm SICS1 NAICSS1
gvkey  fyear                                 
001004 2002   2003   358.411987  5088  423860
       2002   2003   130.628006  3559  333298
       2002   2003    93.415001  4581  488190
       2002   2003    17.386999  5088  423860
       2003   2004   349.527008  5088  423860
...            ...          ...   ...     ...
282189 2006   2006    47.112000  1311  211111
287882 2006   2006  6447.313965  2911  324110
       2006   2006  3205.165039  1311  211111
       2006   2006   411.873993  4612  486110
       2006   2006     0.000000  1311    2111

[63980 rows x 4 columns]

#### Firm vertical relatedness from 2002 to 2006

In [822]:
dffirmvr0206 = dffirmvr(dfss0206, df02VR, df02naics) 

In [823]:
dffirmvr0206

firm vertical relatedness
gvkey  fyear                           
001004 2002                    0.019639
       2002                    0.019639
       2002                    0.019639
       2002                    0.019639
       2003                    0.020303
...                                 ...
282189 2006                    0.000000
287882 2006                    0.303320
       2006                    0.303320
       2006                    0.303320
       2006                    0.303320

[63980 rows x 1 columns]

In [824]:
dffirmvr0206.to_csv('firmvr0206a.csv')

### IO Table 2007

In [466]:
df07 = pd.read_csv('IO2007/IOUse2007.csv')

In [467]:
df07

,Code,1111A0,1111B0,111200,111300,111400,111900,112120,1121A0,112300,...,F06N00,F06S00,F07C00,F07E00,F07N00,F07S00,F10C00,F10E00,F10N00,F10S00
0,1111A0,2420.0,140.0,7.0,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1111B0,NaN,2580.0,NaN,NaN,NaN,133.0,1041.0,2002.0,988.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111200,NaN,NaN,804.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111300,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,111400,NaN,NaN,NaN,NaN,1694.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,S00300,103.0,218.0,NaN,NaN,NaN,NaN,95.0,169.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,S00900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,V00100,231.0,2250.0,2648.0,3656.0,5709.0,4209.0,2314.0,799.0,1720.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,V00200,710.0,-8420.0,245.0,429.0,310.0,-359.0,615.0,1305.0,627.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [468]:
df07.set_index('Code', inplace=True)

In [469]:
df07 = df07.stack().reset_index().rename(columns={"Code": 'Commodity I-O code', 'level_1':'Industry I-O code', 0: 'input'})

In [470]:
df07

,Commodity I-O code,Industry I-O code,input
0,1111A0,1111A0,2420.0
1,1111A0,1111B0,140.0
2,1111A0,111200,7.0
3,1111A0,111900,0.0
4,1111A0,311221,137.0
...,...,...,...
61835,V00300,GSLGH,11886.0
61836,V00300,GSLGO,82210.0
61837,V00300,S00201,-28825.0
61838,V00300,S00202,6759.0


In [825]:
df07VR = indvr(df07)

In [826]:
df07VR

vertical relatedness
Commodity I-O code Industry I-O code                      
1111A0             1111A0                         0.098606
                   1111B0                         0.001350
                   111200                         0.000187
                   111300                         0.000000
                   111400                         0.000000
...                                                    ...
V00300             S00201                        -1.222538
                   S00202                         0.111752
                   S00203                         0.131912
                   S00500                         0.117814
                   S00600                         0.132395

[173400 rows x 1 columns]

### NAICS-IO Conversion Table 2007&2012

**note: no standalone NAICS-IO Conversion Table for 2007 is provided. NAICS-IO 2007 is converted from NAICS-IO 2012 based on the conversion table from U.S. Bureau of Labor Statistics https://www.bls.gov/ces/naics/)**

In [520]:
#import naics-io 2012 table 
df12naics = pd.read_csv('IO2007/IO-NAICS2012.csv', dtype={'I-O Industry number': str,'NAICS':str  })
# delete space in strings
df12naics['I-O Industry number'] = df12naics['I-O Industry number'].apply(str).apply(lambda x: x.strip())
df12naics['NAICS'] = df12naics['NAICS'].apply(str).apply(lambda x: x.strip())
df12naics.set_index('NAICS',inplace=True) #set index

In [521]:
df12naics

,I-O Industry number,I-O Industry title
NAICS,,
11111,1111A0,Oilseed farming
11112,1111A0,Oilseed farming
11113,1111B0,Grain farming
11114,1111B0,Grain farming
11115,1111B0,Grain farming
...,...,...
8133,813A00,"Grantmaking, giving, and social advocacy organ..."
8134,813B00,"Civic, social, professional, and similar organ..."
8139,813B00,"Civic, social, professional, and similar organ..."


In [510]:
#import NAICS conversion table 2012 vs 2007
dfna12to07 = pd.read_csv('IO2007/NAICS2012to2007.csv', dtype={'2012 NAICS Code': str,'2007 NAICS Code':str  })

In [565]:
dfna12to07

,2012 NAICS Code,2007 NAICS Code
0,111110,111110
1,111120,111120
2,111130,111130
3,111140,111140
4,111150,111150
...,...,...
1179,926140,926140
1180,926150,926150
1181,927110,927110
1182,928110,928110


In [557]:
# find different NAICS 2007 codes and add its IO industry to dataframe
dfnaics07md = pd.DataFrame()
for index, row in dfna12to07.iterrows():
    if row['2012 NAICS Code'] != row['2007 NAICS Code']:
        if row['2012 NAICS Code'] in df12naics.index:
            newrow = df12naics.loc[[row['2012 NAICS Code']],:].copy().rename(index={row['2012 NAICS Code']:row['2007 NAICS Code'] })
            dfnaics07md = pd.concat([dfnaics07md, newrow])
        elif row['2012 NAICS Code'][:5] in df12naics.index:
            newrow = df12naics.loc[[row['2012 NAICS Code'][:5]],:].copy().rename(index={row['2012 NAICS Code'][:5]:row['2007 NAICS Code']})
            dfnaics07md = pd.concat([dfnaics07md, newrow])

In [558]:
dfnaics07md

,I-O Industry number,I-O Industry title
NAICS,,
311222,311224,Soybean and other oilseed processing
311223,311224,Soybean and other oilseed processing
311822,3118A0,"Cookie, cracker, pasta, and tortilla manufactu..."
311823,3118A0,"Cookie, cracker, pasta, and tortilla manufactu..."
314121,314120,Curtain and linen mills
...,...,...
339944,339940,Office supplies (except paper) manufacturing
722110,722110,Full-service restaurants
722211,722211,Limited-service restaurants


In [566]:
df07naics = pd.concat([df12naics,dfnaics07md ])

In [567]:
df07naics

,I-O Industry number,I-O Industry title
NAICS,,
11111,1111A0,Oilseed farming
11112,1111A0,Oilseed farming
11113,1111B0,Grain farming
11114,1111B0,Grain farming
11115,1111B0,Grain farming
...,...,...
339944,339940,Office supplies (except paper) manufacturing
722110,722110,Full-service restaurants
722211,722211,Limited-service restaurants


#### Firms' vertical relatedness based on sales 07-11

In [475]:
dfss0711 = dfnaics[(dfnaics['fyear']>=2007) & (dfnaics['fyear']<=2011)].copy()
dfss0711.set_index(['gvkey', 'fyear'], inplace=True)

In [827]:
dffirmvr0711= dffirmvr(dfss0711, df07VR, df07naics) 

In [828]:
dffirmvr0711

firm vertical relatedness
gvkey  fyear                           
001004 2007                    0.006461
       2007                    0.006461
       2007                    0.006461
       2007                    0.006461
       2008                    0.006020
...                                 ...
311524 2011                    0.093003
       2011                    0.093003
       2011                    0.093003
316056 2011                    0.000000
317264 2011                    0.000000

[54309 rows x 1 columns]

In [829]:
#export to csv
dffirmvr0711.to_csv('firmvr0711a.csv')

### IO table 2012

In [591]:
df12 = pd.read_csv('IO2012/IOUse2012.csv')

In [592]:
df12.set_index('Code', inplace=True)

In [593]:
df12 = df12.stack().reset_index().rename(columns={"Code": 'Commodity I-O code', 'level_1':'Industry I-O code', 0: 'input'})

In [594]:
df12

,Commodity I-O code,Industry I-O code,input
0,1111A0,1111A0,2221.0
1,1111A0,1111B0,113.0
2,1111A0,111200,3.0
3,1111A0,111900,0.0
4,1111A0,311221,81.0
...,...,...,...
61839,V00300,GSLGH,16828.0
61840,V00300,GSLGO,95027.0
61841,V00300,S00201,-28222.0
61842,V00300,S00202,15141.0


In [830]:
df12VR = indvr(df12)

In [597]:
df12VR 

vertical relatedness
Commodity I-O code Industry I-O code                      
1111A0             1111A0                         0.052412
                   1111B0                         0.000672
                   111200                         0.000104
                   111900                         0.003174
                   311221                         0.003144
...                                                    ...
V00300             S00201                        -0.957782
                   S00202                         0.126892
                   S00203                         0.131902
                   S00500                         0.117999
                   S00600                         0.128250

[61844 rows x 1 columns]

In [831]:
df12VR 

vertical relatedness
Commodity I-O code Industry I-O code                      
1111A0             1111A0                         0.052412
                   1111B0                         0.000672
                   111200                         0.000104
                   111300                         0.000000
                   111400                         0.000000
...                                                    ...
V00300             S00201                        -0.957782
                   S00202                         0.126892
                   S00203                         0.131902
                   S00500                         0.117999
                   S00600                         0.128250

[173400 rows x 1 columns]

### Firms' vertical relatedness based on sales 12-16

In [598]:
dfss1216 = dfnaics[(dfnaics['fyear']>=2012) & (dfnaics['fyear']<=2016)].copy()
dfss1216.set_index(['gvkey', 'fyear'], inplace=True)

In [832]:
dffirmvr1216= dffirmvr(dfss1216, df12VR, df12naics) 

In [833]:
dffirmvr1216

firm vertical relatedness
gvkey  fyear                           
001004 2012                    0.002158
       2012                    0.002158
       2013                    0.002158
       2013                    0.002158
       2014                    0.002158
...                                 ...
327451 2016                    0.000079
       2016                    0.000079
328795 2016                    0.031076
       2016                    0.031076
       2016                    0.031076

[51740 rows x 1 columns]

In [834]:
dffirmvr1216.to_csv('firmvr1216a.csv')

### Firms' vertical relatedness based on sales 17-19
**Note: the benchmark IO table for 2017 is not available. Use benchmark IO 2007 table to estimate firms' vertical relatedness based on sales from year 2017 to 2019.**

In [606]:
dfss1719 = dfnaics[(dfnaics['fyear']>=2017) & (dfnaics['fyear']<=2019)].copy()
dfss1719.set_index(['gvkey', 'fyear'], inplace=True)

In [836]:
dffirmvr1719 = dffirmvr(dfss1719, df12VR, df12naics)

In [838]:
dffirmvr1719.to_csv('firmvr1719a.csv')

In [837]:
dffirmvr1719

firm vertical relatedness
gvkey  fyear                           
001004 2017                    0.002158
       2017                    0.002158
       2018                    0.002158
       2018                    0.002158
       2019                    0.002158
...                                 ...
330227 2019                    0.000000
331856 2019                    0.000000
332115 2019                    0.000000
335466 2018                    0.000000
       2019                    0.000000

[27314 rows x 1 columns]

## Test 92~96 firms' vertical relatedness based on NAICS

In [861]:
dfnaics

year      salesm SICS1 NAICSS1
gvkey  fyear                                
001004 1992   1993  382.779999  5088  421860
       1993   1994  407.753998  5088  421860
       1994   1995  451.394989  5088  421860
       1995   1996  504.989990  5088  421860
       1996   1997  589.328003  5088  421860
...            ...         ...   ...     ...
330227 2019   2019    5.968000  4412  483111
331856 2019   2019    0.000000  2836  325414
332115 2019   2019    0.000000  2836  325414
335466 2018   2018   20.653999  4499  532411
       2019   2019   41.813000  4499  532411

[332711 rows x 4 columns]

In [875]:
dfnaics.reset_index(inplace = True)
dfnaics[(dfnaics['fyear']>=1992) & (dfnaics['fyear']<=1996)]['NAICSS1'].unique()


array(['421860', '332111', '336510', ..., '422820', '112120', '114119'],
      dtype=object)

In [865]:
len(dfnaics[(dfnaics['fyear']>=1992) & (dfnaics['fyear']<=1996)]['NAICSS1'].unique())

1147

In [876]:
dfss9296n = dfnaics[(dfnaics['fyear']>=1992) & (dfnaics['fyear']<=1996)].copy()

dfss9296n.set_index(['gvkey', 'fyear'], inplace=True)

dfss9296n

year      salesm SICS1 NAICSS1
gvkey  fyear                                
001004 1992   1993  382.779999  5088  421860
       1993   1994  407.753998  5088  421860
       1994   1995  451.394989  5088  421860
       1995   1996  504.989990  5088  421860
       1996   1997  589.328003  5088  421860
...            ...         ...   ...     ...
223007 1996   1996   46.756001  7372  511210
       1996   1996   15.533000  1389  213112
277918 1994   1994    5.787000  3711  336111
       1995   1995   11.625000  3711  336111
       1996   1996    4.209000  3711  336111

[61307 rows x 4 columns]

In [903]:
dfss9296n.dropna(inplace=True)

In [904]:
dfss9296n

year      salesm SICS1 NAICSS1
gvkey  fyear                                
001004 1992   1993  382.779999  5088  421860
       1993   1994  407.753998  5088  421860
       1994   1995  451.394989  5088  421860
       1995   1996  504.989990  5088  421860
       1996   1997  589.328003  5088  421860
...            ...         ...   ...     ...
223007 1996   1996   46.756001  7372  511210
       1996   1996   15.533000  1389  213112
277918 1994   1994    5.787000  3711  336111
       1995   1995   11.625000  3711  336111
       1996   1996    4.209000  3711  336111

[61303 rows x 4 columns]

In [905]:
dffirmvr9296n = dffirmvr(dfss9296n, df97VR, df97naics)

In [908]:
dffirmvr9296n.groupby('fyear').agg({"firm vertical relatedness":np.average})

,firm vertical relatedness
fyear,
1992,0.010442
1993,0.010672
1994,0.010479
1995,0.009787
1996,0.009700


In [909]:
dffirmvr9296n.to_csv('firmvr9296n.csv')

## Combine firm vertical relatedness from 1992 to 2019

In [914]:
dffirmvr9219n = pd.concat([dffirmvr9296n, dffirmvr9701,dffirmvr0206, dffirmvr0711, dffirmvr1216, dffirmvr1719])

In [915]:
dffirmvr9219n = dffirmvr9219n.sort_index()

In [917]:
dffirmvr9219n.to_csv('firmvr9219n.csv')

In [9]:
dffirmvr9219n.drop_duplicates().groupby('fyear').agg({"firm vertical relatedness":np.average})

,firm vertical relatedness
fyear,
1992,0.004892
1993,0.004998
1994,0.004832
1995,0.004439
1996,0.004397
1997,0.005033
1998,0.009662
1999,0.010822
2000,0.010069
